In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import date
payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
#print(first_table.head())
sip=np.array(payload.iloc[:,0])
df=pd.DataFrame()
#for i in range(0,len(sip)):
exp='2022-08-19'
product_type='ETF'

def scrape_weekly_equities():
    url = 'https://www.cboe.com/available_weeklys/'
    weeklys = pd.read_html(url)[1]                                                                                            
    #print(df)
    df.to_csv('test.csv')
    return(weeklys)
#get_weekly_equities()

def read_weekly_equities(product_type=None): 
    weeklys = pd.read_csv('weeklysmf.csv')
    #print(weeklys)
    
    if product_type == 'ETF': 
         weeklys= weeklys[weeklys.Type=='ETF']
            
    if product_type == 'Equity':
        weeklys= weeklys[weeklys.Type=='Equity']

    return(weeklys.iloc[:,0])

option_products=read_weekly_equities(product_type)

#symbol='XBI'
def get_atm_option_data(symbol):
        ticker= yf.Ticker(symbol)
        last_price=ticker.history(period=exp).Close[0]
        calls=ticker.option_chain(date=exp).calls
        puts=ticker.option_chain(date=exp).puts
        atm_call=calls[calls.inTheMoney==True].iloc[-1,:]
        atm_put=puts[puts.inTheMoney==False].iloc[-1,:]
        strike=atm_put.strike
        atm_call_last=(atm_call.bid+atm_call.ask)/2
        atm_put_last=(atm_put.bid+atm_put.ask)/2
        atm_call_IV=atm_call.impliedVolatility
        atm_put_IV=atm_put.impliedVolatility
        atm_data=[symbol,last_price,strike, atm_call_last,atm_put_last,atm_call_IV,atm_put_IV]
        #print(atm_call)
        #print(atm_put)
        #print(atm_call_last)
        #print(atm_put_last)
        ##print(last_price)
        #print(atm_call_IV)
        return(atm_data)

arr=[]
for product in option_products:
    try:
        product_data=get_atm_option_data(product)
        arr.append(product_data)
    except:
        print('pass')
        
df=pd.DataFrame(data=arr,columns=['symbol','last_price','strike', 'atm_call_last','atm_put_last','atm_call_IV','atm_put_IV'])
df.to_csv('option_data_'+exp+"_"+str(date.today())+'.csv')

